In [2]:
        pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

In [4]:
# Include all the nessacary libaries 
from PIL import Image
import os
import PIL
import matplotlib
from matplotlib import pyplot as plt
import glob
from skimage.color import rgb2gray
from skimage.io import imshow, imread, imsave
from skimage.filters import threshold_triangle
from skimage.filters import threshold_mean
from skimage.filters import threshold_otsu
from skimage.morphology import closing
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rescale, resize, downscale_local_mean
import cv2
import numpy as np
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from math import sqrt
from statistics import mean
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [5]:
# Perform Image Processing and Feature Extraction 

def ImageProcessingandFeatureExtraction(Listdir, WriteDir, Threshold):
    
    figure_size = 9 # the dimension of the x and y axis of the kernal.

    listdir = Listdir
    image_path_list = os.listdir(listdir)
    df = pd.DataFrame()
    
    for i in range(len(image_path_list)):
        
        image_path = image_path_list[i]

        img = cv2.imread(listdir + image_path)
        res = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)


        # Pre-process the image
        # Define a function for box filter
        def gaussian_kernel(size, sigma=1):
            size = int(size) // 2
            x, y = np.mgrid[-size:size + 1, -size:size + 1]
            normal = 1 / (2.0 * np.pi * sigma ** 2)
            g = np.exp(-((x ** 2 + y ** 2) / (2.0 * sigma ** 2))) * normal
            return g


        size = 5

        # Apply the gaussian blur
        image = cv2.filter2D(res, -1, gaussian_kernel(size, sigma=1))

        image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # The declaration of CLAHE
        # clipLimit -> Threshold for contrast limiting
        clahe = cv2.createCLAHE(clipLimit=1)
        final_img = clahe.apply(image_bw) + 0


        # Segmentation
        
        binary = final_img < Threshold(final_img)
        binary = closing(binary)
        label_img = label(binary)
        label_img = label_img

        pd.set_option('display.max_columns', None)

        properties = ['convex_area', 'area', 'eccentricity',
                          'extent', 'inertia_tensor',
                          'major_axis_length', 'minor_axis_length',
                          'perimeter', 'solidity',
                          'orientation', 'moments_central',
                          'moments_hu', 'euler_number',
                          'equivalent_diameter',
                          'mean_intensity', 'bbox']

        table = pd.DataFrame(regionprops_table(label_img, image,
                                                   properties=properties))

        real_images = []
        std = []
        mean = []
        percent25 = []
        percent75 = []
        for prop in regionprops(label_img):
            min_row, min_col, max_row, max_col = prop.bbox
            img = image[min_row:max_row, min_col:max_col]
            real_images += [img]
            mean += [np.mean(img)]
            std += [np.std(img)]
            percent25 += [np.percentile(img, 25)]
            percent75 += [np.percentile(img, 75)]
        table['mean_intensity'] = mean
        table['std_intensity'] = std
        table['25th Percentile'] = mean
        table['75th Percentile'] = std
        table['iqr'] = table['75th Percentile'] - table['25th Percentile']
        table['label'] = image_path[5]
        df = pd.concat([df, table], axis=0)

        imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')

    df = df.drop('label', 1)

    # Scale data befor applying PCA
    scaling=StandardScaler()
    # Use fit and transform method
    scaling.fit(df)
    Scaled_data=scaling.transform(df)
    
    # define min max scaler
    scaler = MinMaxScaler()
    # transform data
    scaled = scaler.fit_transform(Scaled_data)

    return scaled

Iterate through all digit folders and apply the Image Processing and Feature Extraction

In [6]:
ReadDir = "Dataset/0/"
WriteDir = "PreprocessedImages/0/"
ZeroTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_mean)
ZeroTable = pd.DataFrame(ZeroTable)
ZeroLabel = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ZeroTable['Label'] = ZeroLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/0/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/0/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/0/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [7]:
ReadDir = "Dataset/1/"
WriteDir = "PreprocessedImages/1/"
OneTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_triangle)
OneTable = pd.DataFrame(OneTable)
OneLabel = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
OneTable['Label'] = OneLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/1/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/1/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/1/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [8]:
ReadDir = "Dataset/2/"
WriteDir = "PreprocessedImages/2/"
TwoTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
TwoTable = pd.DataFrame(TwoTable)
TwoLabel = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
TwoTable['Label'] = TwoLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/2/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/2/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/2/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [9]:
ReadDir = "Dataset/3/"
WriteDir = "PreprocessedImages/3/"
ThreeTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
ThreeTable = pd.DataFrame(ThreeTable)
ThreeLabel = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
ThreeTable['Label'] = ThreeLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/3/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/3/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/3/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [10]:
ReadDir = "Dataset/4/"
WriteDir = "PreprocessedImages/4/"
FourTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
FourTable = pd.DataFrame(FourTable)
FourLabel = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
FourTable['Label'] = FourLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/4/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/4/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/4/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [11]:
ReadDir = "Dataset/5/"
WriteDir = "PreprocessedImages/5/"
FiveTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_mean)
FiveTable = pd.DataFrame(FiveTable)
FiveLabel = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
FiveTable['Label'] = FiveLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/5/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/5/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/5/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [12]:
ReadDir = "Dataset/6/"
WriteDir = "PreprocessedImages/6/"
SixTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
SixTable = pd.DataFrame(SixTable)
SixLabel = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
SixTable['Label'] = SixLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/6/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/6/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/6/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [13]:
ReadDir = "Dataset/7/"
WriteDir = "PreprocessedImages/7/"
SevenTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
SevenTable = pd.DataFrame(SevenTable)
SevenLabel = [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
SevenTable['Label'] = SevenLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/7/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/7/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/7/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [14]:
ReadDir = "Dataset/8/"
WriteDir = "PreprocessedImages/8/"
EightTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
EightTable = pd.DataFrame(EightTable)
EightLabel = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
EightTable['Label'] = EightLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/8/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/8/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/8/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [15]:
ReadDir = "Dataset/9/"
WriteDir = "PreprocessedImages/9/"
NineTable = ImageProcessingandFeatureExtraction(ReadDir, WriteDir, threshold_otsu)
NineTable = pd.DataFrame(NineTable)
NineLabel = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
NineTable['Label'] = NineLabel

<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/9/Preprocessed Image - 0.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/9/Preprocessed Image - 1.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-5-1a801992be5b>:84: UserWarning: PreprocessedImages/9/Preprocessed Image - 2.jpg is a low contrast image
  imsave(WriteDir + os.path.basename("Preprocessed Image - " + str(i)) + '.jpg', label_img, cmap='binary')
Lossy conversion from int32 to uint8. Range [0, 1]. Convert image to uint8 prior to s

In [16]:
# Join all the dataset

JoinDataFrames = [ZeroTable, OneTable, TwoTable, ThreeTable, FourTable, FiveTable, SixTable, SevenTable, EightTable, NineTable]

FinalDataFrame = pd.concat(JoinDataFrames)

In [17]:
# Shuffle all the dataset and view it 
DataFrame = shuffle(FinalDataFrame)
DataFrame

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,Label
9,0.275885,0.326316,1.000000,0.558668,0.088744,0.634104,0.634104,0.455813,0.510981,0.159199,0.323035,0.496527,0.486383,0.326316,0.701923,0.085719,0.927636,0.556604,0.458014,0.060302,0.810834,0.307176,0.868535,0.097657,0.964108,0.430002,0.682435,0.040353,0.835134,0.672306,0.736192,0.043246,0.002132,0.025289,0.081337,0.160214,0.0,0.365423,0.792774,0.792774,0.792774,0.647059,0.800000,0.935484,0.086957,0.455876,0.669537,0.455876,0.669537,0.622819,3
2,0.404060,0.439759,0.735187,0.139568,0.166206,0.774555,0.774555,0.840251,0.855612,0.316217,0.466716,0.420550,0.035516,0.439759,0.609589,0.142697,0.906706,0.251366,0.205355,0.118423,0.096062,0.681300,0.236573,0.139258,0.156016,0.908904,0.188564,0.052750,0.032675,0.855163,0.643132,0.885273,0.721499,0.589062,0.616528,0.474147,1.0,0.465463,0.403727,0.403727,0.403727,0.147059,0.413793,0.533333,0.615385,0.907392,0.000000,0.907392,0.000000,0.035985,9
11,0.464371,0.614583,0.404123,0.206543,1.000000,0.000000,0.000000,0.082605,0.238825,0.749365,0.609211,0.504795,1.000000,0.614583,0.000000,0.889501,1.000000,0.088398,1.000000,0.767728,1.000000,0.132153,0.666617,0.383027,0.972229,0.727945,0.784914,0.642970,0.548767,0.214279,0.057851,0.490404,0.012322,0.000000,0.000000,0.584455,0.0,0.643195,0.358270,0.358270,0.358270,0.641026,0.448276,0.222222,1.000000,0.698634,0.468475,0.698634,0.468475,0.389818,2
10,0.134907,0.228070,0.897498,0.809810,0.091047,0.404406,0.404406,0.146706,0.179778,0.154409,0.240435,0.779624,0.695511,0.228070,0.697115,0.073728,0.947058,0.528302,0.630912,0.046724,0.848268,0.118597,0.821852,0.061037,0.950596,0.496133,0.790828,0.021305,0.866552,0.232792,0.357649,0.072602,0.000000,0.023917,0.042253,0.170210,0.0,0.260563,0.798531,0.798531,0.798531,0.441176,0.950000,0.161290,0.347826,0.226304,0.835740,0.226304,0.835740,0.829612,3
3,0.340065,0.386861,0.000000,0.159598,0.582807,0.452252,0.452252,0.142513,0.325203,0.592057,0.353871,0.249463,1.000000,0.386861,1.000000,0.360556,1.000000,0.720121,0.460808,0.678908,0.326312,0.122785,0.260675,0.067983,0.699854,0.434774,0.402668,0.837891,0.229378,0.451259,0.000000,0.359055,0.041700,0.000000,0.000000,0.706727,0.0,0.446360,0.000000,0.000000,0.000000,0.565217,0.200000,0.047619,0.765957,0.733376,0.421117,0.733376,0.421117,0.339595,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0.845937,0.947846,0.977921,0.097072,0.830407,0.045271,0.045271,0.816470,0.858897,0.829639,0.811759,0.114713,0.598867,0.947846,0.695761,0.841256,0.701819,0.197929,0.940345,0.587492,0.903013,0.817369,0.404915,0.736890,0.320309,0.582533,0.850080,0.666049,0.939651,0.790898,0.741036,0.169471,0.016664,0.007066,0.110003,0.466452,0.0,0.952843,0.854849,0.854849,0.854849,0.075000,0.419355,0.850000,0.400000,1.000000,0.000000,1.000000,0.000000,0.000000,5
8,0.515708,0.587983,0.776814,0.567908,0.236603,0.615649,0.615649,0.545409,0.603752,0.313210,0.623474,0.437139,0.394889,0.587983,0.000000,0.255680,0.765692,0.334746,0.413959,1.000000,0.214993,0.478705,0.125983,0.457874,0.099035,0.861597,0.607316,1.000000,0.427917,0.624257,0.505605,0.000000,1.000000,0.512929,1.000000,0.325092,0.0,0.616644,0.582590,0.582590,0.582590,0.725000,0.851852,1.000000,1.000000,0.549557,0.514300,0.549557,0.514300,0.467637,8
14,0.952010,1.000000,0.663776,0.379588,0.566489,0.595239,0.595239,1.000000,1.000000,0.687939,0.976971,0.200156,0.370793,1.000000,0.000000,0.748994,0.141991,1.000000,0.392537,0.757704,0.536091,1.000000,0.590879,0.550366,0.254662,0.638873,0.314998,0.643643,0.607585,0.802011,0.556362,0.226830,0.099310,0.012035,0.074495,0.065498,0.0,1.000000,0.839898,0.839898,0.839898,0.153846,0.184211,1.000000,0.208333,0.776353,0.092002,0.776353,0.092002,0.166155,6
5,0.164221,0.281179,0.506369,0.758451,0.443030,0.1

In [18]:
# Split the dataset into X and y
X = (DataFrame.iloc[:, 0:50].values)
Y = DataFrame.iloc[:, 50].values

In [19]:
X

array([[0.27588533, 0.32631579, 1.        , ..., 0.4558763 , 0.66953686,
        0.6228188 ],
       [0.40406025, 0.43975904, 0.73518694, ..., 0.90739176, 0.        ,
        0.03598529],
       [0.46437055, 0.61458333, 0.40412273, ..., 0.69863437, 0.46847512,
        0.38981841],
       ...,
       [0.95201038, 1.        , 0.66377633, ..., 0.77635276, 0.09200233,
        0.16615541],
       [0.16422122, 0.28117914, 0.50636932, ..., 0.29777679, 0.65953691,
        0.68163553],
       [0.50244798, 0.60583942, 0.48415229, ..., 0.82380335, 0.23791586,
        0.20534814]])

In [20]:
# ANN class definition 

class ANN(object):

    def __init__(self, data_matrix = None, target_variable = None, hidden = None,
                 iteration_limit = None, learning_rate = None):
        # Create design matrix
        self.N = data_matrix.shape[0]  # assumes df is np array
        design_matrix = pd.DataFrame(data_matrix)
        design_matrix.insert(0, 'bias', np.ones(self.N))  # add column of 1's
        self.X = design_matrix
        
        # Helper variables
        D = data_matrix.shape[1]  # number of dimensions of data matrix
        self.T = pd.get_dummies(target_variable)  # one-hot encoded matrix
        K = len(self.T.columns)  # number of classes
        M = hidden  # number of chosen hidden layers
        self.Y = target_variable  # target variable
        self.tau_max = iteration_limit
        self.eta = learning_rate
        
        # Initialize random weight matrices
        self.W_1 = pd.DataFrame(0.01 * np.random.random(((D + 1), M)))
        self.W_2 = pd.DataFrame(0.01 * np.random.random(((M + 1), K)))
    
    def tanh(self, x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
    
    def forward_propogation(self):
        self.A_1 = np.dot(self.X, self.W_1)
        self.H = self.tanh(self.A_1)
        self.Z_1 = np.insert(self.H, 0, 1, axis=1)  # add column of 1's for bias
        self.A_2 = np.dot(self.Z_1, self.W_2)
        self.Yhat = self.softmax(self.A_2)
        return self
    
    def loss_function(self):
        # calculate the loss for a current iteration
        return sum(np.sum(self.T * np.log(self.Yhat), axis=1))
    
    def accuracy(self):
        # check the prediction accuracy for a current iteration
        return sum(pd.DataFrame(self.Yhat).idxmax(axis=1) == self.Y) / self.N

    def backprop(self):
        # calculate gradient for W_2
        self.delta_k = self.Yhat - self.T  # a notation borrowed from Bishop's text (Yhat - T)
        self.W_2_gradient = np.dot(np.transpose(self.Z_1), self.delta_k)
       
        # calculate gradient for W_1
        self.H_prime = 1 - self.H**2  # derivative of tanh
        self.W_2_reduced = self.W_2.iloc[1:,]  # drop first row from W_2
        self.W_1_gradient = np.dot(np.transpose(self.X),
                                   (self.H_prime * np.dot(self.delta_k,
                                                          np.transpose(self.W_2_reduced))))

        # update weights with gradient descent
        self.W_1 = self.W_1 - self.eta * self.W_1_gradient
        self.W_2 = self.W_2 - self.eta * self.W_2_gradient
        return self

    def train(self):
        for i in range(int(self.tau_max)):
            self.forward_propogation()
            self.backprop()
            if (i % 1000) == 0:
                print('Iteration: ', i,
                      'Loss: ', round(self.loss_function(), 4),
                      'Accuracy: ', round(self.accuracy(), 4))
        

In [21]:
# Complie the Artificial Neural Network 
Network = ANN(data_matrix=X, target_variable=Y,
                        hidden=5, iteration_limit=15000, learning_rate=0.0001)

In [22]:
# Run the training 
Network.train()

Iteration:  0 Loss:  -345.3924 Accuracy:  0.1133
Iteration:  1000 Loss:  -345.2454 Accuracy:  0.16
Iteration:  2000 Loss:  -341.7133 Accuracy:  0.32
Iteration:  3000 Loss:  -309.523 Accuracy:  0.3467
Iteration:  4000 Loss:  -256.3681 Accuracy:  0.5
Iteration:  5000 Loss:  -213.3705 Accuracy:  0.58
Iteration:  6000 Loss:  -178.6655 Accuracy:  0.6933
Iteration:  7000 Loss:  -149.6726 Accuracy:  0.7533
Iteration:  8000 Loss:  -126.6036 Accuracy:  0.8067
Iteration:  9000 Loss:  -109.2134 Accuracy:  0.8333
Iteration:  10000 Loss:  -95.8839 Accuracy:  0.8533
Iteration:  11000 Loss:  -85.0258 Accuracy:  0.8933
Iteration:  12000 Loss:  -75.8762 Accuracy:  0.8933
Iteration:  13000 Loss:  -68.2743 Accuracy:  0.9067
Iteration:  14000 Loss:  -61.8763 Accuracy:  0.9067


In [23]:
# Calculate accuracy percentage between two lists
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Print the accuracy 
accuracy = accuracy_metric(Y, pd.DataFrame(Network.Yhat).idxmax(axis=1))
print(accuracy)

92.0


In [24]:
# Calculate mean absolute error
def mae_metric(actual, predicted):
	sum_error = 0.0
	for i in range(len(actual)):
		sum_error += abs(predicted[i] - actual[i])
	return sum_error / float(len(actual))

mae = mae_metric(Y, pd.DataFrame(Network.Yhat).idxmax(axis=1))
print(mae)

0.28


In [25]:
# Caculate a confusion matrix
def confusion_matrix(actual, predicted):
	unique = set(actual)
	matrix = [list() for x in range(len(unique))]
	for i in range(len(unique)):
		matrix[i] = [0 for x in range(len(unique))]
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for i in range(len(actual)):
		x = lookup[actual[i]]
		y = lookup[predicted[i]]
		matrix[y][x] += 1
	return unique, matrix

In [26]:
# Print Confusion Matrix 
def print_confusion_matrix(unique, matrix):
	print('(A)' + ' '.join(str(x) for x in unique))
	print('(P)---')
	for i, x in enumerate(unique):
		print("%s| %s" % (x, ' '.join(str(x) for x in matrix[i])))
        
unique, matrix = confusion_matrix(Y, pd.DataFrame(Network.Yhat).idxmax(axis=1))
print_confusion_matrix(unique, matrix)

(A)0 1 2 3 4 5 6 7 8 9
(P)---
0| 14 0 0 1 0 0 0 0 0 0
1| 0 14 0 0 0 0 0 1 0 0
2| 0 0 14 0 0 0 0 1 1 0
3| 0 1 0 14 0 0 0 0 0 0
4| 0 0 0 0 14 0 0 0 0 0
5| 0 0 0 0 0 15 0 0 0 0
6| 1 0 0 0 0 0 15 1 0 1
7| 0 0 1 0 1 0 0 11 1 0
8| 0 0 0 0 0 0 0 1 13 0
9| 0 0 0 0 0 0 0 0 0 14
